# Introduction

L'entraînement des réseaux de neurones est long, ce qui m'a amené à chercher pourquoi ils sont aussi longs (et à les améliorer en conséquence). Je me suis aperçu que la lenteur principale venait de l'itération des données et donc de pytorch (des dataloaders) à cause de la grande quantité de données que j'avais (plus ou moins 10 millions). Il y a une première partie sur la lenteur de la libarrie plyfile, une autre partie sur les dataloaders et enfin une partie sur le parallélisme avec torch.multiprocessing.

# Lecture fichier ply

temps : python > np.loadtxt > plyfile

memoire : np.loadtxt > plyfile > python

Donc on utilisera seulement plyfile pour écrire des fichiers .ply puisqu'il se trouve qu'il n'est pas adapté pour lire nos fichiers ply (on lit que les 3 premières colonnes),ce qui donne un temps de lecture très lent par fichier. 

In [1]:
!pip install plyfile

In [2]:
from plyfile import PlyData, PlyElement
from tqdm import tqdm
import time
import numpy as np

with open('/kaggle/input/abc-dataset-train/'+'0002.ply', 'rb') as f:
    st = time.time()
    plydata = PlyData.read(f)
    et = time.time()-st
    print(et)
    
with open('/kaggle/input/abc-dataset-train/'+'0002.ply', 'r') as f:    
    
    st = time.time()
    while "end_header" not in f.readline():
        pass      
    np.loadtxt(f, dtype=np.float32, ndmin=2, usecols=(0, 1, 2))
    et = time.time()-st
    print(et)

0.7907719612121582
0.02049088478088379


# Dataset

Définissons un dataset similaire à celui utilisé dans le notebook "reseau_simple", qui occupe environ 0,3go en mémoire.

In [3]:
import torch
from torch.utils.data import Dataset

In [4]:
class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx],self.labels[idx]
        return sample


n=5000000
data = torch.randn(n, 33)  
labels = torch.randint(0, 1, (n,))  

custom_dataset = CustomDataset(data, labels)

# Dataloader

Définissons un dataloader sur ce dataset. On utilisera la librarie tqdm pour avoir le temps.

In [5]:
from torch.utils.data import DataLoader
from tqdm import tqdm

batch_size = 100
dataloader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=False)

for inputs, labels in tqdm(dataloader):
    pass         

100%|██████████| 50000/50000 [00:32<00:00, 1543.59it/s]


32s pour parcourir 0,3go est long, puisque je rappelle que les données sont sur la ram et que même si c'était sur mon disque dur il fait facilement du 0,3go minimum à la lecture ou en écriture. On en déduit que ce n'est pas une limite sur les E/S (et on peut le constater par exemple en utilisant iostat pour voir l'utilisation du disque dur).

La limite provient de la vitesse d'un seul CPU puisque les autres ne sont pas utilisés. 

Par la suite on va essayer de réduire ces 32s en jouant sur les différents paramètres offerts par le dataloader qui devraient utiliser le reste des CPUs disponibles.

In [6]:
import torch.multiprocessing as mp

print(mp.cpu_count())

4


In [7]:
batch_size = 100
dataloader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

for inputs, labels in tqdm(dataloader):
    pass

100%|██████████| 50000/50000 [01:53<00:00, 440.25it/s]


Ce qui est étonant c'est que c'est moins performant, alors que num_workers=2 permet de dédier 2 cpus au chargement des données.

In [8]:
batch_size = 100
dataloader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=False, num_workers=2, prefetch_factor=100)

for inputs, labels in tqdm(dataloader):
    pass

100%|██████████| 50000/50000 [02:16<00:00, 365.30it/s]


Et en changeant prefetch_factor l'itération devient encore plus lente. Augmenter prefetch_factor permet d'indiquer qu'il faut charger plus de données par CPU (à l'avance).

# Dataloader from scratch

Si on regarde le code source de dataloader (https://github.com/pytorch/pytorch/blob/main/torch/utils/data/dataloader.py), on constate que le shuffle se fait grâce à torch.randperm et que le batch se fait de cette manière avec torch.utils.data.default_collate([x for i in batch]). On peut essayer de reproduire le code :

In [9]:
batch_size = 100

indices = torch.randperm(len(dataloader.dataset))
for i in tqdm(range(0,len(indices),batch_size)):
    data = []
    targets = []
    
    for j in range(i,i+batch_size):
        if j < len(indices):
            data.append(dataloader.dataset.data[indices[j]])
            targets.append(dataloader.dataset.labels[indices[j]])
            
            
    data = torch.utils.data.default_collate(data)
    targets = torch.utils.data.default_collate(targets)   

100%|██████████| 50000/50000 [01:17<00:00, 648.61it/s]


On obtient des performances assez mauvaises comparés à un dataloader. Donc le problème (la lenteur) doit provenir de l'accès aux éléments du dataset. Ce qui est le cas puisqu'en regardant toujours le lien (qui a été envoyé) on observe cette partie du code :

![image.png](a.png)

data = [self.dataset[idx] for idx in possibly_batched_index]

Le dataloaader ne prend pas en compte que les données sont contigues et fera forcément un self.dataset[idx] sur tous les éléments. En reprenant le code d'en-dessous et en supprimant le shuffle on peut arriver à ça : 

In [10]:
for i in tqdm(range(0,len(dataloader.dataset), batch_size)):
    data = dataloader.dataset.data[i*batch_size:(i+1)*batch_size]
    targets = dataloader.dataset.labels[i*batch_size:(i+1)*batch_size]

100%|██████████| 50000/50000 [00:00<00:00, 135869.11it/s]


Ce qui donne des résultats très intéressants puisque contraitement à un dataloader (avec le shuffle désactivé) il continuera d'accèder au dataset de cette manière self.dataset[idx] ce qui est lent.


On pourrait refaire un dataloader en utilisant un accès par "tranche" (slicing) :

In [19]:
import math
    
batch_size = 100

indices = torch.randperm(len(dataloader.dataset))
for i in tqdm(range(0, len(indices), batch_size)):
    end_index = min(i + batch_size, len(indices))
    batch_indices = indices[i:end_index]

    data = dataloader.dataset.data[batch_indices]
    targets = dataloader.dataset.labels[batch_indices]

100%|██████████| 50000/50000 [00:01<00:00, 25616.76it/s]


Ce qui nous donne de bien meilleurs performances. On passe de 32s à 1s.

# ConcatDataset & dataset multiprocessing

J'avais fait une version du chargement des données parallélisés en répartissant les fichiers à charger sur différents process. Le chargement des données était plus rapide, mais j'ai remarqué que le dataloader était plus long. Je pensais que c'était peut-être une question de cache donc j'avais fait un dataset avec les données contigues sauf que ça ne changeait rien. Et effectivement c'est bien un problème : https://github.com/pytorch/pytorch/issues/51011 .

Le code en-dessous pour la parallélisation des données et pour son utilisation se référer à l'un des 2 notebooks.

In [13]:
import torch.multiprocessing as mp

def diviser_liste(liste, nombre):
    taille_sous_liste = max(1,len(liste)//nombre)
    return [liste[i:i + taille_sous_liste] for i in range(0, len(liste), taille_sous_liste)]


def wrapper_charger_datasets(chemin,fichiers, queue):
    queue.put(charger_datasets(chemin,fichiers))

def parallele_charger_datasets(chemin, fichiers, num_cpus):    
    resultat_queue = mp.Manager().Queue()
    partage = diviser_liste(fichiers, num_cpus)

    processes = []
    for sous_liste in partage:
        p = mp.Process(target=wrapper_charger_datasets, args=(chemin, sous_liste, resultat_queue))
        processes.append(p)
        p.start()

    for p in processes:
        p.join()

    list_data = []
    list_labels = []
    while not resultat_queue.empty():
        res = resultat_queue.get()
        for (a,b) in res:
            list_data.append(a)    
            list_labels.append(torch.tensor(b))    
    data = torch.cat(list_data, dim=0)
    labels = torch.cat(list_labels, dim=0)
    concatenated_dataset = SimpleDataset(data,labels)
    
    return concatenated_dataset

# Keras/Tensorflow ?

Pour avoir refait "réseau simple" sous keras/tensorflow j'ai retrouvé le même problème que sur pytorch sur la lenteur des parcours de données, puisqu'on pouvait espèrer que ça aille plus vite car keras/tensorflow utilise des numpy et que les numpy sont plus rapides à parcourir que les tensors. Voir le notebook "reseau_simple avec keras".

In [14]:
test = np.zeros(10000000, dtype=np.float32)

start_time = time.time()
for i in range(10000000):
    test[i]
print(time.time()-start_time, "s")


test2 = torch.from_numpy(test)#on obtient la même chose qu'en passant par torch.zeros

start_time = time.time()
for i in range(10000000):
    test2[i]
print(time.time()-start_time, "s")

2.7042860984802246 s
22.557496309280396 s


On notera aussi une utilisation importance de la mémoire pour créer un itérateur sur un torch : https://github.com/pytorch/pytorch/issues/71266

# Conclusion

L'utilisation d'un dataloader pour le dataset de validation a été suppprimé étant donné sa lenteur, je parcours directement le dataset manuellement. Et pour profiter de la contiguité des données j'ai dû définir mon propre dataloader. 

Pour ce qui est des chargements des données, j'ai supprimé la parallélisation étant donné le gain peu important comparé aux restes (quelques mins) et des problèmes possibles à cause du contexte jupyter-notebook.